# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.10/6912/1 Dashboard: http://10.1.0.10:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1055,Alice,-0.8809549865149464,0.9313687721086061
2000-01-01 00:00:01,943,Patricia,0.9357698678950424,-0.3962682580847814
2000-01-01 00:00:02,960,Frank,-0.33017936530007463,-0.11821503964641789
2000-01-01 00:00:03,987,Norbert,-0.02897710250845864,0.9278211177233568
2000-01-01 00:00:04,1030,Ray,0.4323590243201192,0.850008026184847
2000-01-01 00:00:05,1025,Alice,0.6425392681290045,0.22325949570135695
2000-01-01 00:00:06,982,Charlie,0.6041144467632622,-0.5251232119295934
2000-01-01 00:00:07,998,Yvonne,-0.11925354039473013,0.7345711087418381
2000-01-01 00:00:08,975,Xavier,-0.697437162116356,0.14490096814068631


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1030,Jerry,0.6747545750249477,-0.5154094952069457
2000-01-30 00:00:01,1015,Laura,0.9496657076075676,-0.04595454783496922
2000-01-30 00:00:02,1028,Xavier,-0.4485730715280545,0.39486990269470157
2000-01-30 00:00:03,965,Xavier,0.70234150376389,0.6776310635880165
2000-01-30 00:00:04,975,Zelda,0.3829260040396425,0.7361950798656991
2000-01-30 00:00:05,1002,Norbert,0.7168669960644436,-0.9888563017508094
2000-01-30 00:00:06,1028,Yvonne,-0.9875187525756082,-0.931000550278213
2000-01-30 00:00:07,977,Zelda,-0.2399032895561568,0.3727977030588885
2000-01-30 00:00:08,1044,George,-0.07397696257748199,-0.16166860794269944


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1055,Alice,-0.880955,0.931369
1,2000-01-01 00:00:01,943,Patricia,0.935770,-0.396268
2,2000-01-01 00:00:02,960,Frank,-0.330179,-0.118215
3,2000-01-01 00:00:03,987,Norbert,-0.028977,0.927821
4,2000-01-01 00:00:04,1030,Ray,0.432359,0.850008


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1055,Alice,-0.880955,0.931369
1,2000-01-01 00:00:01,943,Patricia,0.935770,-0.396268
2,2000-01-01 00:00:02,960,Frank,-0.330179,-0.118215
3,2000-01-01 00:00:03,987,Norbert,-0.028977,0.927821
4,2000-01-01 00:00:04,1030,Ray,0.432359,0.850008


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 3.88 s, sys: 489 ms, total: 4.36 s
Wall time: 4.98 s


name
Alice       0.000376
Bob        -0.002012
Charlie    -0.002091
Dan        -0.000019
Edith      -0.000020
Frank      -0.001315
George     -0.001260
Hannah      0.001640
Ingrid     -0.001936
Jerry       0.000249
Kevin       0.002209
Laura       0.003978
Michael     0.002059
Norbert    -0.000161
Oliver     -0.000092
Patricia    0.001976
Quinn       0.001807
Ray        -0.001864
Sarah      -0.001103
Tim         0.001039
Ursula      0.000623
Victor      0.000403
Wendy      -0.001168
Xavier      0.005640
Yvonne      0.000676
Zelda       0.002132
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/pyarrow/compat.py:24: FutureWarning: pyarrow.compat has been deprecated and will be removed in a future release
  warnings.warn("pyarrow.compat has been deprecated and will be removed in a "


In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.18 s, sys: 168 ms, total: 1.35 s
Wall time: 1.31 s


name
Alice       0.000376
Bob        -0.002012
Charlie    -0.002091
Dan        -0.000019
Edith      -0.000020
Frank      -0.001315
George     -0.001260
Hannah      0.001640
Ingrid     -0.001936
Jerry       0.000249
Kevin       0.002209
Laura       0.003978
Michael     0.002059
Norbert    -0.000161
Oliver     -0.000092
Patricia    0.001976
Quinn       0.001807
Ray        -0.001864
Sarah      -0.001103
Tim         0.001039
Ursula      0.000623
Victor      0.000403
Wendy      -0.001168
Xavier      0.005640
Yvonne      0.000676
Zelda       0.002132
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.02 s, sys: 109 ms, total: 1.13 s
Wall time: 1.42 s


name
Alice       0.000376
Bob        -0.002012
Charlie    -0.002091
Dan        -0.000019
Edith      -0.000020
Frank      -0.001315
George     -0.001260
Hannah      0.001640
Ingrid     -0.001936
Jerry       0.000249
Kevin       0.002209
Laura       0.003978
Michael     0.002059
Norbert    -0.000161
Oliver     -0.000092
Patricia    0.001976
Quinn       0.001807
Ray        -0.001864
Sarah      -0.001103
Tim         0.001039
Ursula      0.000623
Victor      0.000403
Wendy      -0.001168
Xavier      0.005640
Yvonne      0.000676
Zelda       0.002132
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html